In [11]:
# date: 4/28/2022
# author: Dog Yang
# task: got the validation for stays after weidong did these things
import os, sys, gc
import pandas as pd
import numpy as np
import yaml
import os
import glob
#import ipystata

from google.cloud import storage
from google.cloud import bigquery

path = os.path.expanduser('~')
# set up env credential variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path + '/Dropbox/Amenity/source/analysis/veraset_gravy_gps_sample/firm-exit-3608acd14b06.json'
client = bigquery.Client()

import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [46]:
#validtaion4，Redding，do not run,5.2日的select

namelist = ['05-01']
for x in namelist:
    
    default_dataset = 'firm-exit.test_chongrui_redding_4'

    job_config = bigquery.QueryJobConfig(default_dataset = default_dataset)

query_label = f'''
    Create or replace table validation_'''+ x + f''' as (
    with check_for_unique as
    (select 
        -- take necessary variables and conversions
        distinct PDT_time, caid, location_name,
        date(PDT_time) as date
        FROM `firm-exit.veraset_tripchain.visitation_ca`),  select_near as
    (select*,
        -- 取前后两天的数据，加上当天一共三天
      from check_for_unique where PDT_time >= '2019-05-01 00:00:00' and PDT_time < '2019-06-30 00:00:00'),
    --计算每个caid在每一次访问中距离每一个整点的时间(这里考虑到跨天的情况)，并取绝对值
    get_date_max as (select caid, date,location_name, PDT_time,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 00:00:00' AS TIMESTAMP),CAST(PDT_time AS TIMESTAMP),MINUTE) as close_0,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 01:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_1,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 02:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_2,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 03:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_3,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 04:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_4,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 05:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_5,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 06:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_6,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 07:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_7,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 08:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_8,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 09:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_9,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 10:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_10,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 11:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_11,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 12:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_12,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 13:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_13,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 14:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_14,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 15:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_15,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 16:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_16,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 17:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_17,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 18:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_18,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 19:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_19,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 20:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_20,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 21:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_21,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 22:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_22,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 23:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_23
    from select_near),
    
    
    cancel_zero_1 as (SELECT * FROM get_date_max where close_1 >= 0),
    get_1 as (SELECT tb1.* FROM cancel_zero_1 tb1 INNER JOIN (SELECT caid,MIN(close_1) 
    minclose_1 FROM cancel_zero_1 GROUP BY caid) tbb1 ON tbb1.caid = tb1.caid 
    WHERE tbb1.minclose_1 = tb1.close_1),
    set_1_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_1
        from get_1),
        
    cancel_zero_2 as (SELECT * FROM get_date_max where close_2 >= 0),
    get_2 as (SELECT tb2.* FROM cancel_zero_2 tb2 INNER JOIN (SELECT caid,MIN(close_2) 
    minclose_2 FROM cancel_zero_2 GROUP BY caid) tbb2 ON tbb2.caid = tb2.caid 
    WHERE tbb2.minclose_2 = tb2.close_2),
    set_2_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_2
        from get_2),
    
    
    cancel_zero_3 as (SELECT * FROM get_date_max where close_3 >= 0),
    get_3 as (SELECT tb3.* FROM cancel_zero_3 tb3 INNER JOIN (SELECT caid,MIN(close_3) 
    minclose_3 FROM cancel_zero_3 GROUP BY caid) tbb3 ON tbb3.caid = tb3.caid 
    WHERE tbb3.minclose_3 = tb3.close_3),
    set_3_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_3
        from get_3),
    
    cancel_zero_4 as (SELECT * FROM get_date_max where close_4 >= 0),
    get_4 as (SELECT tb4.* FROM cancel_zero_4 tb4 INNER JOIN (SELECT caid,MIN(close_4) 
    minclose_4 FROM cancel_zero_4 GROUP BY caid) tbb4 ON tbb4.caid = tb4.caid 
    WHERE tbb4.minclose_4 = tb4.close_4),
    set_4_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_4
        from get_4),
    
    cancel_zero_5 as (SELECT * FROM get_date_max where close_5 >= 0),
    get_5 as (SELECT tb5.* FROM cancel_zero_5 tb5 INNER JOIN (SELECT caid,MIN(close_5) 
    minclose_5 FROM cancel_zero_5 GROUP BY caid) tbb5 ON tbb5.caid = tb5.caid 
    WHERE tbb5.minclose_5 = tb5.close_5),
    set_5_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_5
        from get_5),
    
    cancel_zero_6 as (SELECT * FROM get_date_max where close_6 >= 0),
    get_6 as (SELECT tb6.* FROM cancel_zero_6 tb6 INNER JOIN (SELECT caid,MIN(close_6) 
    minclose_6 FROM cancel_zero_6 GROUP BY caid) tbb6 ON tbb6.caid = tb6.caid 
      WHERE tbb6.minclose_6 = tb6.close_6),
    set_6_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_6
        from get_6),
        
    cancel_zero_7 as (SELECT * FROM get_date_max where close_7 >= 0),
    get_7 as (SELECT tb7.* FROM cancel_zero_7 tb7 INNER JOIN (SELECT caid,MIN(close_7) 
    minclose_7 FROM cancel_zero_7 GROUP BY caid) tbb7 ON tbb7.caid = tb7.caid 
    WHERE tbb7.minclose_7 = tb7.close_7),
    set_7_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_7
        from get_7),
    
    cancel_zero_8 as (SELECT * FROM get_date_max where close_8 >= 0),
    get_8 as (SELECT tb8.* FROM cancel_zero_8 tb8 INNER JOIN (SELECT caid,MIN(close_8) 
    minclose_8 FROM cancel_zero_8 GROUP BY caid) tbb8 ON tbb8.caid = tb8.caid 
    WHERE tbb8.minclose_8 = tb8.close_8),
    set_8_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_8
        from get_8),
    
    cancel_zero_9 as (SELECT * FROM get_date_max where close_9 >= 0),
    get_9 as (SELECT tb9.* FROM cancel_zero_9 tb9 INNER JOIN (SELECT caid,MIN(close_9) 
    minclose_9 FROM cancel_zero_9 GROUP BY caid) tbb9 ON tbb9.caid = tb9.caid 
    WHERE tbb9.minclose_9 = tb9.close_9),
    set_9_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_9
        from get_9),

    cancel_zero_10 as (SELECT * FROM get_date_max where close_10 >= 0),
    get_10 as (SELECT tb10.* FROM cancel_zero_10 tb10 INNER JOIN (SELECT caid,MIN(close_10) 
    minclose_10 FROM cancel_zero_10 GROUP BY caid) tbb10 ON tbb10.caid = tb10.caid 
    WHERE tbb10.minclose_10 = tb10.close_10),
    set_10_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_10
        from get_10),
    
    cancel_zero_11 as (SELECT * FROM get_date_max where close_11 >= 0),
    get_11 as (SELECT tb11.* FROM  cancel_zero_11 tb11 INNER JOIN (SELECT caid,MIN(close_11) 
    minclose_11 FROM  cancel_zero_11 GROUP BY caid) tbb11 ON tbb11.caid = tb11.caid 
    WHERE tbb11.minclose_11 = tb11.close_11),
    set_11_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_11
        from get_11),
        
    cancel_zero_12 as (SELECT * FROM get_date_max where close_12 >= 0),
    get_12 as (SELECT tb12.* FROM cancel_zero_12 tb12 INNER JOIN (SELECT caid,MIN(close_12) 
    minclose_12 FROM cancel_zero_12 GROUP BY caid) tbb12 ON tbb12.caid = tb12.caid 
    WHERE tbb12.minclose_12 = tb12.close_12),
    set_12_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_12
        from get_12),
        
    cancel_zero_13 as (SELECT * FROM get_date_max where close_13 >= 0),
    get_13 as (SELECT tb13.* FROM cancel_zero_13 tb13 INNER JOIN (SELECT caid,MIN(close_13) 
    minclose_13 FROM cancel_zero_13 GROUP BY caid) tbb13 ON tbb13.caid = tb13.caid 
    WHERE tbb13.minclose_13 = tb13.close_13),
    set_13_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_13
        from get_13),
        
    cancel_zero_14 as (SELECT * FROM get_date_max where close_14 >= 0),
    get_14 as (SELECT tb14.* FROM cancel_zero_14 tb14 INNER JOIN (SELECT caid,MIN(close_14) 
    minclose_14 FROM cancel_zero_14 GROUP BY caid) tbb14 ON tbb14.caid = tb14.caid
    WHERE tbb14.minclose_14 = tb14.close_14),
    set_14_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_14
        from get_14),
        
    cancel_zero_15 as (SELECT * FROM get_date_max where close_15 >= 0),
    get_15 as (SELECT tb15.* FROM cancel_zero_15 tb15 INNER JOIN (SELECT caid,MIN(close_15) 
    minclose_15 FROM cancel_zero_15 GROUP BY caid) tbb15 ON tbb15.caid = tb15.caid 
    WHERE tbb15.minclose_15 = tb15.close_15),
    set_15_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_15
        from get_15),
        
     cancel_zero_16 as (SELECT * FROM get_date_max where close_16 >= 0),
    get_16 as (SELECT tb16.* FROM cancel_zero_16 tb16 INNER JOIN (SELECT caid,MIN(close_16) 
    minclose_16 FROM cancel_zero_16 GROUP BY caid) tbb16 ON tbb16.caid = tb16.caid 
    WHERE tbb16.minclose_16 = tb16.close_16),
    set_16_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_16
        from get_16),

    cancel_zero_17 as (SELECT * FROM get_date_max where close_17 >= 0),
    get_17 as (SELECT tb17.* FROM cancel_zero_17 tb17 INNER JOIN (SELECT caid,MIN(close_17) 
    minclose_17 FROM cancel_zero_17 GROUP BY caid) tbb17 ON tbb17.caid = tb17.caid 
    WHERE tbb17.minclose_17 = tb17.close_17),
    set_17_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_17
        from get_17),
        
    cancel_zero_18 as (SELECT * FROM get_date_max where close_18 >= 0),
    get_18 as (SELECT tb18.* FROM cancel_zero_18 tb18 INNER JOIN (SELECT caid,MIN(close_18) 
    minclose_18 FROM cancel_zero_18 GROUP BY caid) tbb18 ON tbb18.caid = tb18.caid 
    WHERE tbb18.minclose_18 = tb18.close_18),
    set_18_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_18
        from get_18),

    cancel_zero_19 as (SELECT * FROM get_date_max where close_19 >= 0),
    get_19 as (SELECT tb19.* FROM cancel_zero_19 tb19 INNER JOIN (SELECT caid,MIN(close_19) 
    minclose_19 FROM cancel_zero_19 GROUP BY caid) tbb19 ON tbb19.caid = tb19.caid 
    WHERE tbb19.minclose_19 = tb19.close_19),
    set_19_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_19
        from get_19),
        
    cancel_zero_20 as (SELECT * FROM get_date_max where close_20 >= 0),
    get_20 as (SELECT tb20.* FROM cancel_zero_20 tb20 INNER JOIN (SELECT caid,MIN(close_20) 
    minclose_20 FROM cancel_zero_20 GROUP BY caid) tbb20 ON tbb20.caid = tb20.caid 
    WHERE tbb20.minclose_20 = tb20.close_20),
    set_20_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_20
        from get_20),
        
    cancel_zero_21 as (SELECT * FROM get_date_max where close_21 >= 0),
    get_21 as (SELECT tb21.* FROM cancel_zero_21 tb21 INNER JOIN (SELECT caid,MIN(close_21) 
    minclose_21 FROM cancel_zero_21 GROUP BY caid) tbb21 ON tbb21.caid = tb21.caid 
    WHERE tbb21.minclose_21 = tb21.close_21),
    set_21_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_21
        from get_21),

    cancel_zero_22 as (SELECT * FROM get_date_max where close_22 >= 0),
    get_22 as (SELECT tb22.* FROM cancel_zero_22 tb22 INNER JOIN (SELECT caid,MIN(close_22) 
    minclose_22 FROM cancel_zero_22 GROUP BY caid) tbb22 ON tbb22.caid = tb22.caid 
    WHERE tbb22.minclose_22 = tb22.close_22),
    set_22_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_22
        from get_22),
        
    cancel_zero_23 as (SELECT * FROM get_date_max where close_23 >= 0),
    get_23 as (SELECT tb23.* FROM cancel_zero_23 tb23 INNER JOIN (SELECT caid,MIN(close_23) 
    minclose_23 FROM cancel_zero_23 GROUP BY caid) tbb23 ON tbb23.caid = tb23.caid 
    WHERE tbb23.minclose_23 = tb23.close_23),
    set_23_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_23
        from get_23),
        
    cancel_zero_0 as (SELECT * FROM get_date_max where close_0 >= 0),
    get_0 as (SELECT tb0.* FROM cancel_zero_0 tb0 INNER JOIN (SELECT caid,MIN(close_0) 
    minclose_0 FROM cancel_zero_0 GROUP BY caid ) tbb0 ON tbb0.caid = tb0.caid 
    WHERE tbb0.minclose_0 = tb0.close_0),
    set_0_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_0
        from get_0),
        
       union_all as 
        ---选择每个caid每天分别离24个小时最近的记录，也就是每个id的二十四个状态，将其合并，增添一些赋值为xx的空行方便合并
    (select caid, date, number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_0_min
    UNION ALL 
    select caid, date, "xx" as number_0, number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_1_min
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_2_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2,number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_3_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_4_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_5_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_6_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_7_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_8_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_9_min)
    UNION ALL
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_10_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_11_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_12_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_13_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_14_min)
    UNION ALL 
    (select caid, date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_15_min
    UNION ALL 
    select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_16_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_17_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_18_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_19_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_20_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, number_21, "xx" as number_22, "xx" as number_23 from set_21_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, number_22, "xx" as number_23 from set_22_min)
    UNION ALL 
    (select caid,date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, number_23 from set_23_min
    ))
    
 ---计算所有人在所有天里，在0点中分别是在家，在工作和Other的状态，并记录下来，后面同理
    select 
        countif(number_0 = 'home') as num_records_home_0,
        countif(number_0 = 'work') as num_records_work_0, 
        countif(number_0 = 'other') as num_records_other_0,
        countif(number_1 = 'home') as num_records_home_1,
        countif(number_1 = 'work') as num_records_work_1, 
        countif(number_1 = 'other') as num_records_other_1,
        countif(number_2 = 'home') as num_records_home_2,
        countif(number_2 = 'work') as num_records_work_2, 
        countif(number_2 = 'other') as num_records_other_2,
        countif(number_3 = 'home') as num_records_home_3,
        countif(number_3 = 'work') as num_records_work_3, 
        countif(number_3 = 'other') as num_records_other_3,
        countif(number_4 = 'home') as num_records_home_4,
        countif(number_4 = 'work') as num_records_work_4, 
        countif(number_4 = 'other') as num_records_other_4,
        countif(number_5 = 'home') as num_records_home_5,
        countif(number_5 = 'work') as num_records_work_5, 
        countif(number_5 = 'other') as num_records_other_5,
        countif(number_6 = 'home') as num_records_home_6,
        countif(number_6 = 'work') as num_records_work_6, 
        countif(number_6 = 'other') as num_records_other_6,
        countif(number_7 = 'home') as num_records_home_7,
        countif(number_7 = 'work') as num_records_work_7, 
        countif(number_7 = 'other') as num_records_other_7,
        countif(number_8 = 'home') as num_records_home_8,
        countif(number_8 = 'work') as num_records_work_8, 
        countif(number_8 = 'other') as num_records_other_8,
        countif(number_9 = 'home') as num_records_home_9,
        countif(number_9 = 'work') as num_records_work_9, 
        countif(number_9 = 'other') as num_records_other_9,
        countif(number_10 = 'home') as num_records_home_10,
        countif(number_10 = 'work') as num_records_work_10, 
        countif(number_10 = 'other') as num_records_other_10,
        countif(number_11 = 'home') as num_records_home_11,
        countif(number_11 = 'work') as num_records_work_11, 
        countif(number_11 = 'other') as num_records_other_11,
        countif(number_12 = 'home') as num_records_home_12,
        countif(number_12 = 'work') as num_records_work_12, 
        countif(number_12 = 'other') as num_records_other_12,
        countif(number_13 = 'home') as num_records_home_13,
        countif(number_13 = 'work') as num_records_work_13, 
        countif(number_13 = 'other') as num_records_other_13,
        countif(number_14 = 'home') as num_records_home_14,
        countif(number_14 = 'work') as num_records_work_14, 
        countif(number_14 = 'other') as num_records_other_14,
        countif(number_15 = 'home') as num_records_home_15,
        countif(number_15 = 'work') as num_records_work_15, 
        countif(number_15 = 'other') as num_records_other_15,
        countif(number_16 = 'home') as num_records_home_16,
        countif(number_16 = 'work') as num_records_work_16, 
        countif(number_16 = 'other') as num_records_other_16,
        countif(number_17 = 'home') as num_records_home_17,
        countif(number_17 = 'work') as num_records_work_17, 
        countif(number_17 = 'other') as num_records_other_17,
        countif(number_18 = 'home') as num_records_home_18,
        countif(number_18 = 'work') as num_records_work_18, 
        countif(number_18 = 'other') as num_records_other_18,
        countif(number_19 = 'home') as num_records_home_19,
        countif(number_19 = 'work') as num_records_work_19, 
        countif(number_19 = 'other') as num_records_other_19,
        countif(number_20 = 'home') as num_records_home_20,
        countif(number_20 = 'work') as num_records_work_20, 
        countif(number_20 = 'other') as num_records_other_20,
        countif(number_21 = 'home') as num_records_home_21,
        countif(number_21 = 'work') as num_records_work_21, 
        countif(number_21 = 'other') as num_records_other_21,
        countif(number_22 = 'home') as num_records_home_22,
        countif(number_22 = 'work') as num_records_work_22, 
        countif(number_22 = 'other') as num_records_other_22,
        countif(number_23 = 'home') as num_records_home_23,
        countif(number_23 = 'work') as num_records_work_23, 
        countif(number_23 = 'other') as num_records_other_23 from union_all)
    '''  
    
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [238]:
702170

In [108]:
#validtaion4，Redding，do not run,5.2日的select

namelist = ['06-18']
for x in namelist:
    
    default_dataset = 'firm-exit.test_chongrui_redding_4'
    
    job_config = bigquery.QueryJobConfig(default_dataset = default_dataset)

query_label = f'''
    Create or replace table validation_'''+ x + f''' as (
    with check_for_unique as
    (select 
        -- take necessary variables and conversions
        distinct PDT_time, caid, location_name,
        date(PDT_time) as date
        FROM `firm-exit.veraset_tripchain.visitation_ca`),  select_near as
    (select*,
        -- 取前后两天的数据，加上当天一共三天
      from check_for_unique where PDT_time >= '2019-04-30 00:00:00' and PDT_time < '2019-06-30 00:00:00'),
    --计算每个caid在每一次访问中距离每一个整点的时间(这里考虑到跨天的情况)，并取绝对值
    get_date_max as (select caid, date,location_name, PDT_time,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 00:00:00' AS TIMESTAMP),CAST(PDT_time AS TIMESTAMP),MINUTE) as close_0,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 01:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_1,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 02:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_2,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 03:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_3,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 04:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_4,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 05:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_5,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 06:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_6,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 07:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_7,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 08:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_8,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 09:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_9,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 10:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_10,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 11:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_11,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 12:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_12,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 13:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_13,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 14:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_14,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 15:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_15,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 16:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_16,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 17:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_17,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 18:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_18,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 19:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_19,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 20:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_20,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 21:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_21,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 22:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_22,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 23:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_23
    from select_near),
    
    
    cancel_zero_1 as (SELECT * FROM get_date_max where close_1 >= 0),
    get_1 as (SELECT tb1.* FROM cancel_zero_1 tb1 INNER JOIN (SELECT caid,MIN(close_1) 
    minclose_1 FROM cancel_zero_1 GROUP BY caid) tbb1 ON tbb1.caid = tb1.caid 
    WHERE tbb1.minclose_1 = tb1.close_1),
    set_1_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_1
        from get_1),
        
    cancel_zero_2 as (SELECT * FROM get_date_max where close_2 >= 0),
    get_2 as (SELECT tb2.* FROM cancel_zero_2 tb2 INNER JOIN (SELECT caid,MIN(close_2) 
    minclose_2 FROM cancel_zero_2 GROUP BY caid) tbb2 ON tbb2.caid = tb2.caid 
    WHERE tbb2.minclose_2 = tb2.close_2),
    set_2_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_2
        from get_2),
    
    
    cancel_zero_3 as (SELECT * FROM get_date_max where close_3 >= 0),
    get_3 as (SELECT tb3.* FROM cancel_zero_3 tb3 INNER JOIN (SELECT caid,MIN(close_3) 
    minclose_3 FROM cancel_zero_3 GROUP BY caid) tbb3 ON tbb3.caid = tb3.caid 
    WHERE tbb3.minclose_3 = tb3.close_3),
    set_3_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_3
        from get_3),
    
    cancel_zero_4 as (SELECT * FROM get_date_max where close_4 >= 0),
    get_4 as (SELECT tb4.* FROM cancel_zero_4 tb4 INNER JOIN (SELECT caid,MIN(close_4) 
    minclose_4 FROM cancel_zero_4 GROUP BY caid) tbb4 ON tbb4.caid = tb4.caid 
    WHERE tbb4.minclose_4 = tb4.close_4),
    set_4_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_4
        from get_4),
    
    cancel_zero_5 as (SELECT * FROM get_date_max where close_5 >= 0),
    get_5 as (SELECT tb5.* FROM cancel_zero_5 tb5 INNER JOIN (SELECT caid,MIN(close_5) 
    minclose_5 FROM cancel_zero_5 GROUP BY caid) tbb5 ON tbb5.caid = tb5.caid 
    WHERE tbb5.minclose_5 = tb5.close_5),
    set_5_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_5
        from get_5),
    
    cancel_zero_6 as (SELECT * FROM get_date_max where close_6 >= 0),
    get_6 as (SELECT tb6.* FROM cancel_zero_6 tb6 INNER JOIN (SELECT caid,MIN(close_6) 
    minclose_6 FROM cancel_zero_6 GROUP BY caid) tbb6 ON tbb6.caid = tb6.caid 
      WHERE tbb6.minclose_6 = tb6.close_6),
    set_6_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_6
        from get_6),
        
    cancel_zero_7 as (SELECT * FROM get_date_max where close_7 >= 0),
    get_7 as (SELECT tb7.* FROM cancel_zero_7 tb7 INNER JOIN (SELECT caid,MIN(close_7) 
    minclose_7 FROM cancel_zero_7 GROUP BY caid) tbb7 ON tbb7.caid = tb7.caid 
    WHERE tbb7.minclose_7 = tb7.close_7),
    set_7_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_7
        from get_7),
    
    cancel_zero_8 as (SELECT * FROM get_date_max where close_8 >= 0),
    get_8 as (SELECT tb8.* FROM cancel_zero_8 tb8 INNER JOIN (SELECT caid,MIN(close_8) 
    minclose_8 FROM cancel_zero_8 GROUP BY caid) tbb8 ON tbb8.caid = tb8.caid 
    WHERE tbb8.minclose_8 = tb8.close_8),
    set_8_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_8
        from get_8),
    
    cancel_zero_9 as (SELECT * FROM get_date_max where close_9 >= 0),
    get_9 as (SELECT tb9.* FROM cancel_zero_9 tb9 INNER JOIN (SELECT caid,MIN(close_9) 
    minclose_9 FROM cancel_zero_9 GROUP BY caid) tbb9 ON tbb9.caid = tb9.caid 
    WHERE tbb9.minclose_9 = tb9.close_9),
    set_9_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_9
        from get_9),

    cancel_zero_10 as (SELECT * FROM get_date_max where close_10 >= 0),
    get_10 as (SELECT tb10.* FROM cancel_zero_10 tb10 INNER JOIN (SELECT caid,MIN(close_10) 
    minclose_10 FROM cancel_zero_10 GROUP BY caid) tbb10 ON tbb10.caid = tb10.caid 
    WHERE tbb10.minclose_10 = tb10.close_10),
    set_10_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_10
        from get_10),
    
    cancel_zero_11 as (SELECT * FROM get_date_max where close_11 >= 0),
    get_11 as (SELECT tb11.* FROM  cancel_zero_11 tb11 INNER JOIN (SELECT caid,MIN(close_11) 
    minclose_11 FROM  cancel_zero_11 GROUP BY caid) tbb11 ON tbb11.caid = tb11.caid 
    WHERE tbb11.minclose_11 = tb11.close_11),
    set_11_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_11
        from get_11),
        
    cancel_zero_12 as (SELECT * FROM get_date_max where close_12 >= 0),
    get_12 as (SELECT tb12.* FROM cancel_zero_12 tb12 INNER JOIN (SELECT caid,MIN(close_12) 
    minclose_12 FROM cancel_zero_12 GROUP BY caid) tbb12 ON tbb12.caid = tb12.caid 
    WHERE tbb12.minclose_12 = tb12.close_12),
    set_12_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_12
        from get_12),
        
    cancel_zero_13 as (SELECT * FROM get_date_max where close_13 >= 0),
    get_13 as (SELECT tb13.* FROM cancel_zero_13 tb13 INNER JOIN (SELECT caid,MIN(close_13) 
    minclose_13 FROM cancel_zero_13 GROUP BY caid) tbb13 ON tbb13.caid = tb13.caid 
    WHERE tbb13.minclose_13 = tb13.close_13),
    set_13_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_13
        from get_13),
        
    cancel_zero_14 as (SELECT * FROM get_date_max where close_14 >= 0),
    get_14 as (SELECT tb14.* FROM cancel_zero_14 tb14 INNER JOIN (SELECT caid,MIN(close_14) 
    minclose_14 FROM cancel_zero_14 GROUP BY caid) tbb14 ON tbb14.caid = tb14.caid
    WHERE tbb14.minclose_14 = tb14.close_14),
    set_14_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_14
        from get_14),
        
    cancel_zero_15 as (SELECT * FROM get_date_max where close_15 >= 0),
    get_15 as (SELECT tb15.* FROM cancel_zero_15 tb15 INNER JOIN (SELECT caid,MIN(close_15) 
    minclose_15 FROM cancel_zero_15 GROUP BY caid) tbb15 ON tbb15.caid = tb15.caid 
    WHERE tbb15.minclose_15 = tb15.close_15),
    set_15_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_15
        from get_15),
        
     cancel_zero_16 as (SELECT * FROM get_date_max where close_16 >= 0),
    get_16 as (SELECT tb16.* FROM cancel_zero_16 tb16 INNER JOIN (SELECT caid,MIN(close_16) 
    minclose_16 FROM cancel_zero_16 GROUP BY caid) tbb16 ON tbb16.caid = tb16.caid 
    WHERE tbb16.minclose_16 = tb16.close_16),
    set_16_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_16
        from get_16),

    cancel_zero_17 as (SELECT * FROM get_date_max where close_17 >= 0),
    get_17 as (SELECT tb17.* FROM cancel_zero_17 tb17 INNER JOIN (SELECT caid,MIN(close_17) 
    minclose_17 FROM cancel_zero_17 GROUP BY caid) tbb17 ON tbb17.caid = tb17.caid 
    WHERE tbb17.minclose_17 = tb17.close_17),
    set_17_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_17
        from get_17),
        
    cancel_zero_18 as (SELECT * FROM get_date_max where close_18 >= 0),
    get_18 as (SELECT tb18.* FROM cancel_zero_18 tb18 INNER JOIN (SELECT caid,MIN(close_18) 
    minclose_18 FROM cancel_zero_18 GROUP BY caid) tbb18 ON tbb18.caid = tb18.caid 
    WHERE tbb18.minclose_18 = tb18.close_18),
    set_18_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_18
        from get_18),

    cancel_zero_19 as (SELECT * FROM get_date_max where close_19 >= 0),
    get_19 as (SELECT tb19.* FROM cancel_zero_19 tb19 INNER JOIN (SELECT caid,MIN(close_19) 
    minclose_19 FROM cancel_zero_19 GROUP BY caid) tbb19 ON tbb19.caid = tb19.caid 
    WHERE tbb19.minclose_19 = tb19.close_19),
    set_19_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_19
        from get_19),
        
    cancel_zero_20 as (SELECT * FROM get_date_max where close_20 >= 0),
    get_20 as (SELECT tb20.* FROM cancel_zero_20 tb20 INNER JOIN (SELECT caid,MIN(close_20) 
    minclose_20 FROM cancel_zero_20 GROUP BY caid) tbb20 ON tbb20.caid = tb20.caid 
    WHERE tbb20.minclose_20 = tb20.close_20),
    set_20_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_20
        from get_20),
        
    cancel_zero_21 as (SELECT * FROM get_date_max where close_21 >= 0),
    get_21 as (SELECT tb21.* FROM cancel_zero_21 tb21 INNER JOIN (SELECT caid,MIN(close_21) 
    minclose_21 FROM cancel_zero_21 GROUP BY caid) tbb21 ON tbb21.caid = tb21.caid 
    WHERE tbb21.minclose_21 = tb21.close_21),
    set_21_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_21
        from get_21),

    cancel_zero_22 as (SELECT * FROM get_date_max where close_22 >= 0),
    get_22 as (SELECT tb22.* FROM cancel_zero_22 tb22 INNER JOIN (SELECT caid,MIN(close_22) 
    minclose_22 FROM cancel_zero_22 GROUP BY caid) tbb22 ON tbb22.caid = tb22.caid 
    WHERE tbb22.minclose_22 = tb22.close_22),
    set_22_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_22
        from get_22),
        
    cancel_zero_23 as (SELECT * FROM get_date_max where close_23 >= 0),
    get_23 as (SELECT tb23.* FROM cancel_zero_23 tb23 INNER JOIN (SELECT caid,MIN(close_23) 
    minclose_23 FROM cancel_zero_23 GROUP BY caid) tbb23 ON tbb23.caid = tb23.caid 
    WHERE tbb23.minclose_23 = tb23.close_23),
    set_23_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_23
        from get_23),
        
    cancel_zero_0 as (SELECT * FROM get_date_max where close_0 >= 0),
    get_0 as (SELECT tb0.* FROM cancel_zero_0 tb0 INNER JOIN (SELECT caid,MIN(close_0) 
    minclose_0 FROM cancel_zero_0 GROUP BY caid ) tbb0 ON tbb0.caid = tb0.caid 
    WHERE tbb0.minclose_0 = tb0.close_0),
    set_0_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_0
        from get_0),
        
       union_all as 
        ---选择每个caid每天分别离24个小时最近的记录，也就是每个id的二十四个状态，将其合并，增添一些赋值为xx的空行方便合并
    (select caid, date, number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_0_min
    UNION ALL 
    select caid, date, "xx" as number_0, number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_1_min
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_2_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2,number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_3_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_4_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_5_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_6_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_7_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_8_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_9_min)
    UNION ALL
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_10_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_11_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_12_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_13_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_14_min)
    UNION ALL 
    (select caid, date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_15_min
    UNION ALL 
    select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_16_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_17_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_18_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_19_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_20_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, number_21, "xx" as number_22, "xx" as number_23 from set_21_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, number_22, "xx" as number_23 from set_22_min)
    UNION ALL 
    (select caid,date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, number_23 from set_23_min
    ))
    
 ---计算所有人在所有天里，在0点中分别是在家，在工作和Other的状态，并记录下来，后面同理
    select 
        countif(number_0 = 'home') as num_records_home_0,
        countif(number_0 = 'work') as num_records_work_0, 
        countif(number_0 = 'other') as num_records_other_0,
        countif(number_1 = 'home') as num_records_home_1,
        countif(number_1 = 'work') as num_records_work_1, 
        countif(number_1 = 'other') as num_records_other_1,
        countif(number_2 = 'home') as num_records_home_2,
        countif(number_2 = 'work') as num_records_work_2, 
        countif(number_2 = 'other') as num_records_other_2,
        countif(number_3 = 'home') as num_records_home_3,
        countif(number_3 = 'work') as num_records_work_3, 
        countif(number_3 = 'other') as num_records_other_3,
        countif(number_4 = 'home') as num_records_home_4,
        countif(number_4 = 'work') as num_records_work_4, 
        countif(number_4 = 'other') as num_records_other_4,
        countif(number_5 = 'home') as num_records_home_5,
        countif(number_5 = 'work') as num_records_work_5, 
        countif(number_5 = 'other') as num_records_other_5,
        countif(number_6 = 'home') as num_records_home_6,
        countif(number_6 = 'work') as num_records_work_6, 
        countif(number_6 = 'other') as num_records_other_6,
        countif(number_7 = 'home') as num_records_home_7,
        countif(number_7 = 'work') as num_records_work_7, 
        countif(number_7 = 'other') as num_records_other_7,
        countif(number_8 = 'home') as num_records_home_8,
        countif(number_8 = 'work') as num_records_work_8, 
        countif(number_8 = 'other') as num_records_other_8,
        countif(number_9 = 'home') as num_records_home_9,
        countif(number_9 = 'work') as num_records_work_9, 
        countif(number_9 = 'other') as num_records_other_9,
        countif(number_10 = 'home') as num_records_home_10,
        countif(number_10 = 'work') as num_records_work_10, 
        countif(number_10 = 'other') as num_records_other_10,
        countif(number_11 = 'home') as num_records_home_11,
        countif(number_11 = 'work') as num_records_work_11, 
        countif(number_11 = 'other') as num_records_other_11,
        countif(number_12 = 'home') as num_records_home_12,
        countif(number_12 = 'work') as num_records_work_12, 
        countif(number_12 = 'other') as num_records_other_12,
        countif(number_13 = 'home') as num_records_home_13,
        countif(number_13 = 'work') as num_records_work_13, 
        countif(number_13 = 'other') as num_records_other_13,
        countif(number_14 = 'home') as num_records_home_14,
        countif(number_14 = 'work') as num_records_work_14, 
        countif(number_14 = 'other') as num_records_other_14,
        countif(number_15 = 'home') as num_records_home_15,
        countif(number_15 = 'work') as num_records_work_15, 
        countif(number_15 = 'other') as num_records_other_15,
        countif(number_16 = 'home') as num_records_home_16,
        countif(number_16 = 'work') as num_records_work_16, 
        countif(number_16 = 'other') as num_records_other_16,
        countif(number_17 = 'home') as num_records_home_17,
        countif(number_17 = 'work') as num_records_work_17, 
        countif(number_17 = 'other') as num_records_other_17,
        countif(number_18 = 'home') as num_records_home_18,
        countif(number_18 = 'work') as num_records_work_18, 
        countif(number_18 = 'other') as num_records_other_18,
        countif(number_19 = 'home') as num_records_home_19,
        countif(number_19 = 'work') as num_records_work_19, 
        countif(number_19 = 'other') as num_records_other_19,
        countif(number_20 = 'home') as num_records_home_20,
        countif(number_20 = 'work') as num_records_work_20, 
        countif(number_20 = 'other') as num_records_other_20,
        countif(number_21 = 'home') as num_records_home_21,
        countif(number_21 = 'work') as num_records_work_21, 
        countif(number_21 = 'other') as num_records_other_21,
        countif(number_22 = 'home') as num_records_home_22,
        countif(number_22 = 'work') as num_records_work_22, 
        countif(number_22 = 'other') as num_records_other_22,
        countif(number_23 = 'home') as num_records_home_23,
        countif(number_23 = 'work') as num_records_work_23, 
        countif(number_23 = 'other') as num_records_other_23 from union_all)
    '''  
    
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [237]:
default_dataset = 'firm-exit.test'
destination_table = 'firm-exit.test.merge'
job_config = bigquery.QueryJobConfig(write_disposition='WRITE_TRUNCATE', # overwrite
                                     destination=destination_table,
                                     default_dataset = default_dataset)
query_label = f'''
    select *, "501" as date from validation_05-01 
    union all
    select *, "502" as date from validation_05-02
    union all
    select *, "503" as date from validation_05-03
    union all
    select *,"504" as date from validation_05-04
    union all
    select *,"505" as date from validation_05-05
    union all
    select *,"506" as date from validation_05-06
    union all
    select *,"507" as date from validation_05-07
    union all
    select *,"508" as date from validation_05-08
    union all
    select *,"509" as date from validation_05-09
    union all    
    select *,"510" as date from validation_05-10
    union all
    select *,"511" as date from validation_05-11
    union all
    select * ,"512" as date from validation_05-12 
    union all
    select * ,"513" as date from validation_05-13
    union all
    select *,"514" as date from validation_05-14
    union all
    select *,"515" as date from validation_05-15 
    union all
    select *,"516" as date from validation_05-16
    union all
    select *,"517" as date from validation_05-17 
    union all
    select *,"518" as date from validation_05-18 
    union all
    select *,"519" as date from validation_05-19 
    union all
    select *,"520" as date from validation_05-20 
    union all
    select *,"521" as date from validation_05-21 
    union all
    select *,"522" as date from validation_05-22
    union all
    select *,"523" as date from validation_05-23
    union all
    select *,"524" as date from validation_05-24 
    union all
    select *,"525" as date from validation_05-25
    union all
    select *,"526" as date from validation_05-26 
    union all
    select *,"527" as date from validation_05-27 
    union all
    select *,"528" as date from validation_05-28 
    union all
    select *,"529" as date from validation_05-29
    union all
    select *,"530" as date from validation_05-30 
    union all
    select *,"531" as date from validation_05-31 
    union all
    select *,"601" as date from validation_06-01 
    union all
    select *,"602" as date from validation_06-02
    union all
    select *,"603" as date from validation_06-03 
    union all
    select *,"604" as date from validation_06-04 
    union all
    select *,"605" as date from validation_06-05
    union all
    select *,"606" as date from validation_06-06 
    union all
    select *,"607" as date from validation_06-07
    union all
    select *,"608" as date from validation_06-08 
    union all
    select *,"609" as date from validation_06-09
    union all
    select *,"610" as date from validation_06-10
    union all
    select *,"611" as date from validation_06-11
    union all
    select *,"612" as date from validation_06-12 
    union all
    select *,"613" as date from validation_06-13 
    union all
    select *,"614" as date from validation_06-14
    union all
    select *,"615" as date from validation_06-15 
    union all
    select *,"616" as date from validation_06-16 
    union all
    select *,"617" as date from validation_06-17 
    union all
    select *,"618" as date from validation_06-18
    union all
    select *,"619" as date from validation_06-19 
    union all
    select *,"620" as date from validation_06-20
    union all
    select *,"621" as date from validation_06-21 
    union all
    select *,"622" as date from validation_06-22 
    union all
    select *,"623" as date from validation_06-23 
    union all
    select *,"624" as date from validation_06-24
    union all
    select *,"625" as date from validation_06-25
    union all
    select *,"626" as date from validation_06-26 
    union all
    select *,"627" as date from validation_06-27 
    union all
    select *,"628" as date from validation_06-28
    union all
    select *,"629" as date from validation_06-29 
'''
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [175]:
default_dataset = 'firm-exit.test'
destination_table = 'firm-exit.test.cancel'
job_config = bigquery.QueryJobConfig(write_disposition='WRITE_TRUNCATE', # overwrite
                                     destination=destination_table,
                                     default_dataset = default_dataset)
query_label = f'''
with choose as(
select *,
      case
          when (location_name = 'work') then 10000000
          else 0
      end
      as work_mark 
      from `firm-exit.veraset_tripchain.visitation_ca`), select_day_without_work as
    (select *,
     avg(work_mark) OVER (PARTITION BY caid) as avg_mark
     from choose)
    select * from select_day_without_work where avg_mark > 0
'''
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [236]:
#validtaion4，Redding，do not run,5.2日的select

namelist = ['06-29']
for x in namelist:
    
    default_dataset = 'firm-exit.test'
    
    job_config = bigquery.QueryJobConfig(default_dataset = default_dataset)

query_label = f'''
    Create or replace table validation_'''+ x + f''' as (
    with check_for_unique as
    (select 
        -- take necessary variables and conversions
        distinct PDT_time, caid, location_name,
        date(PDT_time) as date
        FROM `firm-exit.test.cancel` where PDT_time >= '2019-06-28 00:00:00' and PDT_time < '2019-06-30 00:00:00'), 
    --计算每个caid在每一次访问中距离每一个整点的时间(这里考虑到跨天的情况)，并取绝对值
    get_date_max as (select caid, date,location_name, PDT_time,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 00:00:00' AS TIMESTAMP),CAST(PDT_time AS TIMESTAMP),MINUTE) as close_0,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 01:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_1,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 02:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_2,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 03:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_3,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 04:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_4,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 05:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_5,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 06:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_6,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 07:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_7,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 08:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_8,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 09:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_9,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 10:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_10,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 11:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_11,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 12:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_12,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 13:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_13,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 14:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_14,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 15:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_15,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 16:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_16,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 17:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_17,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 18:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_18,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 19:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_19,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 20:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_20,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 21:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_21,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 22:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_22,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 23:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_23
    from check_for_unique),
    
    
    cancel_zero_1 as (SELECT * FROM get_date_max where close_1 >= 0),
    get_1 as (SELECT tb1.* FROM cancel_zero_1 tb1 INNER JOIN (SELECT caid,MIN(close_1) 
    minclose_1 FROM cancel_zero_1 GROUP BY caid) tbb1 ON tbb1.caid = tb1.caid 
    WHERE tbb1.minclose_1 = tb1.close_1),
    set_1_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_1
        from get_1),
        
    cancel_zero_2 as (SELECT * FROM get_date_max where close_2 >= 0),
    get_2 as (SELECT tb2.* FROM cancel_zero_2 tb2 INNER JOIN (SELECT caid,MIN(close_2) 
    minclose_2 FROM cancel_zero_2 GROUP BY caid) tbb2 ON tbb2.caid = tb2.caid 
    WHERE tbb2.minclose_2 = tb2.close_2),
    set_2_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_2
        from get_2),
    
    
    cancel_zero_3 as (SELECT * FROM get_date_max where close_3 >= 0),
    get_3 as (SELECT tb3.* FROM cancel_zero_3 tb3 INNER JOIN (SELECT caid,MIN(close_3) 
    minclose_3 FROM cancel_zero_3 GROUP BY caid) tbb3 ON tbb3.caid = tb3.caid 
    WHERE tbb3.minclose_3 = tb3.close_3),
    set_3_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_3
        from get_3),
    
    cancel_zero_4 as (SELECT * FROM get_date_max where close_4 >= 0),
    get_4 as (SELECT tb4.* FROM cancel_zero_4 tb4 INNER JOIN (SELECT caid,MIN(close_4) 
    minclose_4 FROM cancel_zero_4 GROUP BY caid) tbb4 ON tbb4.caid = tb4.caid 
    WHERE tbb4.minclose_4 = tb4.close_4),
    set_4_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_4
        from get_4),
    
    cancel_zero_5 as (SELECT * FROM get_date_max where close_5 >= 0),
    get_5 as (SELECT tb5.* FROM cancel_zero_5 tb5 INNER JOIN (SELECT caid,MIN(close_5) 
    minclose_5 FROM cancel_zero_5 GROUP BY caid) tbb5 ON tbb5.caid = tb5.caid 
    WHERE tbb5.minclose_5 = tb5.close_5),
    set_5_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_5
        from get_5),
    
    cancel_zero_6 as (SELECT * FROM get_date_max where close_6 >= 0),
    get_6 as (SELECT tb6.* FROM cancel_zero_6 tb6 INNER JOIN (SELECT caid,MIN(close_6) 
    minclose_6 FROM cancel_zero_6 GROUP BY caid) tbb6 ON tbb6.caid = tb6.caid 
      WHERE tbb6.minclose_6 = tb6.close_6),
    set_6_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_6
        from get_6),
        
    cancel_zero_7 as (SELECT * FROM get_date_max where close_7 >= 0),
    get_7 as (SELECT tb7.* FROM cancel_zero_7 tb7 INNER JOIN (SELECT caid,MIN(close_7) 
    minclose_7 FROM cancel_zero_7 GROUP BY caid) tbb7 ON tbb7.caid = tb7.caid 
    WHERE tbb7.minclose_7 = tb7.close_7),
    set_7_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_7
        from get_7),
    
    cancel_zero_8 as (SELECT * FROM get_date_max where close_8 >= 0),
    get_8 as (SELECT tb8.* FROM cancel_zero_8 tb8 INNER JOIN (SELECT caid,MIN(close_8) 
    minclose_8 FROM cancel_zero_8 GROUP BY caid) tbb8 ON tbb8.caid = tb8.caid 
    WHERE tbb8.minclose_8 = tb8.close_8),
    set_8_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_8
        from get_8),
    
    cancel_zero_9 as (SELECT * FROM get_date_max where close_9 >= 0),
    get_9 as (SELECT tb9.* FROM cancel_zero_9 tb9 INNER JOIN (SELECT caid,MIN(close_9) 
    minclose_9 FROM cancel_zero_9 GROUP BY caid) tbb9 ON tbb9.caid = tb9.caid 
    WHERE tbb9.minclose_9 = tb9.close_9),
    set_9_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_9
        from get_9),

    cancel_zero_10 as (SELECT * FROM get_date_max where close_10 >= 0),
    get_10 as (SELECT tb10.* FROM cancel_zero_10 tb10 INNER JOIN (SELECT caid,MIN(close_10) 
    minclose_10 FROM cancel_zero_10 GROUP BY caid) tbb10 ON tbb10.caid = tb10.caid 
    WHERE tbb10.minclose_10 = tb10.close_10),
    set_10_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_10
        from get_10),
    
    cancel_zero_11 as (SELECT * FROM get_date_max where close_11 >= 0),
    get_11 as (SELECT tb11.* FROM  cancel_zero_11 tb11 INNER JOIN (SELECT caid,MIN(close_11) 
    minclose_11 FROM  cancel_zero_11 GROUP BY caid) tbb11 ON tbb11.caid = tb11.caid 
    WHERE tbb11.minclose_11 = tb11.close_11),
    set_11_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_11
        from get_11),
        
    cancel_zero_12 as (SELECT * FROM get_date_max where close_12 >= 0),
    get_12 as (SELECT tb12.* FROM cancel_zero_12 tb12 INNER JOIN (SELECT caid,MIN(close_12) 
    minclose_12 FROM cancel_zero_12 GROUP BY caid) tbb12 ON tbb12.caid = tb12.caid 
    WHERE tbb12.minclose_12 = tb12.close_12),
    set_12_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_12
        from get_12),
        
    cancel_zero_13 as (SELECT * FROM get_date_max where close_13 >= 0),
    get_13 as (SELECT tb13.* FROM cancel_zero_13 tb13 INNER JOIN (SELECT caid,MIN(close_13) 
    minclose_13 FROM cancel_zero_13 GROUP BY caid) tbb13 ON tbb13.caid = tb13.caid 
    WHERE tbb13.minclose_13 = tb13.close_13),
    set_13_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_13
        from get_13),
        
    cancel_zero_14 as (SELECT * FROM get_date_max where close_14 >= 0),
    get_14 as (SELECT tb14.* FROM cancel_zero_14 tb14 INNER JOIN (SELECT caid,MIN(close_14) 
    minclose_14 FROM cancel_zero_14 GROUP BY caid) tbb14 ON tbb14.caid = tb14.caid
    WHERE tbb14.minclose_14 = tb14.close_14),
    set_14_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_14
        from get_14),
        
    cancel_zero_15 as (SELECT * FROM get_date_max where close_15 >= 0),
    get_15 as (SELECT tb15.* FROM cancel_zero_15 tb15 INNER JOIN (SELECT caid,MIN(close_15) 
    minclose_15 FROM cancel_zero_15 GROUP BY caid) tbb15 ON tbb15.caid = tb15.caid 
    WHERE tbb15.minclose_15 = tb15.close_15),
    set_15_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_15
        from get_15),
        
     cancel_zero_16 as (SELECT * FROM get_date_max where close_16 >= 0),
    get_16 as (SELECT tb16.* FROM cancel_zero_16 tb16 INNER JOIN (SELECT caid,MIN(close_16) 
    minclose_16 FROM cancel_zero_16 GROUP BY caid) tbb16 ON tbb16.caid = tb16.caid 
    WHERE tbb16.minclose_16 = tb16.close_16),
    set_16_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_16
        from get_16),

    cancel_zero_17 as (SELECT * FROM get_date_max where close_17 >= 0),
    get_17 as (SELECT tb17.* FROM cancel_zero_17 tb17 INNER JOIN (SELECT caid,MIN(close_17) 
    minclose_17 FROM cancel_zero_17 GROUP BY caid) tbb17 ON tbb17.caid = tb17.caid 
    WHERE tbb17.minclose_17 = tb17.close_17),
    set_17_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_17
        from get_17),
        
    cancel_zero_18 as (SELECT * FROM get_date_max where close_18 >= 0),
    get_18 as (SELECT tb18.* FROM cancel_zero_18 tb18 INNER JOIN (SELECT caid,MIN(close_18) 
    minclose_18 FROM cancel_zero_18 GROUP BY caid) tbb18 ON tbb18.caid = tb18.caid 
    WHERE tbb18.minclose_18 = tb18.close_18),
    set_18_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_18
        from get_18),

    cancel_zero_19 as (SELECT * FROM get_date_max where close_19 >= 0),
    get_19 as (SELECT tb19.* FROM cancel_zero_19 tb19 INNER JOIN (SELECT caid,MIN(close_19) 
    minclose_19 FROM cancel_zero_19 GROUP BY caid) tbb19 ON tbb19.caid = tb19.caid 
    WHERE tbb19.minclose_19 = tb19.close_19),
    set_19_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_19
        from get_19),
        
    cancel_zero_20 as (SELECT * FROM get_date_max where close_20 >= 0),
    get_20 as (SELECT tb20.* FROM cancel_zero_20 tb20 INNER JOIN (SELECT caid,MIN(close_20) 
    minclose_20 FROM cancel_zero_20 GROUP BY caid) tbb20 ON tbb20.caid = tb20.caid 
    WHERE tbb20.minclose_20 = tb20.close_20),
    set_20_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_20
        from get_20),
        
    cancel_zero_21 as (SELECT * FROM get_date_max where close_21 >= 0),
    get_21 as (SELECT tb21.* FROM cancel_zero_21 tb21 INNER JOIN (SELECT caid,MIN(close_21) 
    minclose_21 FROM cancel_zero_21 GROUP BY caid) tbb21 ON tbb21.caid = tb21.caid 
    WHERE tbb21.minclose_21 = tb21.close_21),
    set_21_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_21
        from get_21),

    cancel_zero_22 as (SELECT * FROM get_date_max where close_22 >= 0),
    get_22 as (SELECT tb22.* FROM cancel_zero_22 tb22 INNER JOIN (SELECT caid,MIN(close_22) 
    minclose_22 FROM cancel_zero_22 GROUP BY caid) tbb22 ON tbb22.caid = tb22.caid 
    WHERE tbb22.minclose_22 = tb22.close_22),
    set_22_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_22
        from get_22),
        
    cancel_zero_23 as (SELECT * FROM get_date_max where close_23 >= 0),
    get_23 as (SELECT tb23.* FROM cancel_zero_23 tb23 INNER JOIN (SELECT caid,MIN(close_23) 
    minclose_23 FROM cancel_zero_23 GROUP BY caid) tbb23 ON tbb23.caid = tb23.caid 
    WHERE tbb23.minclose_23 = tb23.close_23),
    set_23_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_23
        from get_23),
        
    cancel_zero_0 as (SELECT * FROM get_date_max where close_0 >= 0),
    get_0 as (SELECT tb0.* FROM cancel_zero_0 tb0 INNER JOIN (SELECT caid,MIN(close_0) 
    minclose_0 FROM cancel_zero_0 GROUP BY caid ) tbb0 ON tbb0.caid = tb0.caid 
    WHERE tbb0.minclose_0 = tb0.close_0),
    set_0_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_0
        from get_0),
        
       union_all as 
        ---选择每个caid每天分别离24个小时最近的记录，也就是每个id的二十四个状态，将其合并，增添一些赋值为xx的空行方便合并
    (select caid, date, number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_0_min
    UNION ALL 
    select caid, date, "xx" as number_0, number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_1_min
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_2_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2,number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_3_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_4_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_5_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_6_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_7_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_8_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_9_min)
    UNION ALL
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_10_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_11_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_12_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_13_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_14_min)
    UNION ALL 
    (select caid, date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_15_min
    UNION ALL 
    select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_16_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_17_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_18_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_19_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_20_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, number_21, "xx" as number_22, "xx" as number_23 from set_21_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, number_22, "xx" as number_23 from set_22_min)
    UNION ALL 
    (select caid,date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, number_23 from set_23_min
    ))
    
 ---计算所有人在所有天里，在0点中分别是在家，在工作和Other的状态，并记录下来，后面同理
    select 
        countif(number_0 = 'home') as num_records_home_0,
        countif(number_0 = 'work') as num_records_work_0, 
        countif(number_0 = 'other') as num_records_other_0,
        countif(number_1 = 'home') as num_records_home_1,
        countif(number_1 = 'work') as num_records_work_1, 
        countif(number_1 = 'other') as num_records_other_1,
        countif(number_2 = 'home') as num_records_home_2,
        countif(number_2 = 'work') as num_records_work_2, 
        countif(number_2 = 'other') as num_records_other_2,
        countif(number_3 = 'home') as num_records_home_3,
        countif(number_3 = 'work') as num_records_work_3, 
        countif(number_3 = 'other') as num_records_other_3,
        countif(number_4 = 'home') as num_records_home_4,
        countif(number_4 = 'work') as num_records_work_4, 
        countif(number_4 = 'other') as num_records_other_4,
        countif(number_5 = 'home') as num_records_home_5,
        countif(number_5 = 'work') as num_records_work_5, 
        countif(number_5 = 'other') as num_records_other_5,
        countif(number_6 = 'home') as num_records_home_6,
        countif(number_6 = 'work') as num_records_work_6, 
        countif(number_6 = 'other') as num_records_other_6,
        countif(number_7 = 'home') as num_records_home_7,
        countif(number_7 = 'work') as num_records_work_7, 
        countif(number_7 = 'other') as num_records_other_7,
        countif(number_8 = 'home') as num_records_home_8,
        countif(number_8 = 'work') as num_records_work_8, 
        countif(number_8 = 'other') as num_records_other_8,
        countif(number_9 = 'home') as num_records_home_9,
        countif(number_9 = 'work') as num_records_work_9, 
        countif(number_9 = 'other') as num_records_other_9,
        countif(number_10 = 'home') as num_records_home_10,
        countif(number_10 = 'work') as num_records_work_10, 
        countif(number_10 = 'other') as num_records_other_10,
        countif(number_11 = 'home') as num_records_home_11,
        countif(number_11 = 'work') as num_records_work_11, 
        countif(number_11 = 'other') as num_records_other_11,
        countif(number_12 = 'home') as num_records_home_12,
        countif(number_12 = 'work') as num_records_work_12, 
        countif(number_12 = 'other') as num_records_other_12,
        countif(number_13 = 'home') as num_records_home_13,
        countif(number_13 = 'work') as num_records_work_13, 
        countif(number_13 = 'other') as num_records_other_13,
        countif(number_14 = 'home') as num_records_home_14,
        countif(number_14 = 'work') as num_records_work_14, 
        countif(number_14 = 'other') as num_records_other_14,
        countif(number_15 = 'home') as num_records_home_15,
        countif(number_15 = 'work') as num_records_work_15, 
        countif(number_15 = 'other') as num_records_other_15,
        countif(number_16 = 'home') as num_records_home_16,
        countif(number_16 = 'work') as num_records_work_16, 
        countif(number_16 = 'other') as num_records_other_16,
        countif(number_17 = 'home') as num_records_home_17,
        countif(number_17 = 'work') as num_records_work_17, 
        countif(number_17 = 'other') as num_records_other_17,
        countif(number_18 = 'home') as num_records_home_18,
        countif(number_18 = 'work') as num_records_work_18, 
        countif(number_18 = 'other') as num_records_other_18,
        countif(number_19 = 'home') as num_records_home_19,
        countif(number_19 = 'work') as num_records_work_19, 
        countif(number_19 = 'other') as num_records_other_19,
        countif(number_20 = 'home') as num_records_home_20,
        countif(number_20 = 'work') as num_records_work_20, 
        countif(number_20 = 'other') as num_records_other_20,
        countif(number_21 = 'home') as num_records_home_21,
        countif(number_21 = 'work') as num_records_work_21, 
        countif(number_21 = 'other') as num_records_other_21,
        countif(number_22 = 'home') as num_records_home_22,
        countif(number_22 = 'work') as num_records_work_22, 
        countif(number_22 = 'other') as num_records_other_22,
        countif(number_23 = 'home') as num_records_home_23,
        countif(number_23 = 'work') as num_records_work_23, 
        countif(number_23 = 'other') as num_records_other_23 from union_all)
    '''  
    
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [182]:
#validtaion4，Redding，do not run,5.2日的select

namelist = ['06-05']
for x in namelist:
    
    default_dataset = 'firm-exit.test_chongrui_redding_4_restrict'
    
    job_config = bigquery.QueryJobConfig(default_dataset = default_dataset)

query_label = f'''
    Create or replace table validation_'''+ x + f''' as (
    with check_for_unique as
    (select 
        -- take necessary variables and conversions
        distinct PDT_time, caid, location_name,
        date(PDT_time) as date
        FROM `firm-exit.veraset_tripchain.visitation_ca`),  select_near as
    (select*,
        -- 取前后两天的数据，加上当天一共三天
      from check_for_unique where PDT_time >= '2019-06-04 00:00:00' and PDT_time < '2019-06-30 00:00:00'),
    --计算每个caid在每一次访问中距离每一个整点的时间(这里考虑到跨天的情况)，并取绝对值
    get_date_max as (select caid, date,location_name, PDT_time,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 00:00:00' AS TIMESTAMP),CAST(PDT_time AS TIMESTAMP),MINUTE) as close_0,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 01:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_1,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 02:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_2,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 03:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_3,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 04:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_4,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 05:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_5,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 06:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_6,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 07:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_7,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 08:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_8,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 09:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_9,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 10:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_10,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 11:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_11,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 12:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_12,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 13:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_13,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 14:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_14,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 15:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_15,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 16:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_16,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 17:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_17,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 18:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_18,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 19:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_19,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 20:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_20,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 21:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_21,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 22:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_22,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 23:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_23
    from select_near),
    
    
    cancel_zero_1 as (SELECT * FROM get_date_max where close_1 >= 0),
    get_1 as (SELECT tb1.* FROM cancel_zero_1 tb1 INNER JOIN (SELECT caid,MIN(close_1) 
    minclose_1 FROM cancel_zero_1 GROUP BY caid) tbb1 ON tbb1.caid = tb1.caid 
    WHERE tbb1.minclose_1 = tb1.close_1),
    set_1_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_1
        from get_1),
        
    cancel_zero_2 as (SELECT * FROM get_date_max where close_2 >= 0),
    get_2 as (SELECT tb2.* FROM cancel_zero_2 tb2 INNER JOIN (SELECT caid,MIN(close_2) 
    minclose_2 FROM cancel_zero_2 GROUP BY caid) tbb2 ON tbb2.caid = tb2.caid 
    WHERE tbb2.minclose_2 = tb2.close_2),
    set_2_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_2
        from get_2),
    
    
    cancel_zero_3 as (SELECT * FROM get_date_max where close_3 >= 0),
    get_3 as (SELECT tb3.* FROM cancel_zero_3 tb3 INNER JOIN (SELECT caid,MIN(close_3) 
    minclose_3 FROM cancel_zero_3 GROUP BY caid) tbb3 ON tbb3.caid = tb3.caid 
    WHERE tbb3.minclose_3 = tb3.close_3),
    set_3_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_3
        from get_3),
    
    cancel_zero_4 as (SELECT * FROM get_date_max where close_4 >= 0),
    get_4 as (SELECT tb4.* FROM cancel_zero_4 tb4 INNER JOIN (SELECT caid,MIN(close_4) 
    minclose_4 FROM cancel_zero_4 GROUP BY caid) tbb4 ON tbb4.caid = tb4.caid 
    WHERE tbb4.minclose_4 = tb4.close_4),
    set_4_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_4
        from get_4),
    
    cancel_zero_5 as (SELECT * FROM get_date_max where close_5 >= 0),
    get_5 as (SELECT tb5.* FROM cancel_zero_5 tb5 INNER JOIN (SELECT caid,MIN(close_5) 
    minclose_5 FROM cancel_zero_5 GROUP BY caid) tbb5 ON tbb5.caid = tb5.caid 
    WHERE tbb5.minclose_5 = tb5.close_5),
    set_5_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_5
        from get_5),
    
    cancel_zero_6 as (SELECT * FROM get_date_max where close_6 >= 0),
    get_6 as (SELECT tb6.* FROM cancel_zero_6 tb6 INNER JOIN (SELECT caid,MIN(close_6) 
    minclose_6 FROM cancel_zero_6 GROUP BY caid) tbb6 ON tbb6.caid = tb6.caid 
      WHERE tbb6.minclose_6 = tb6.close_6),
    set_6_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_6
        from get_6),
        
    cancel_zero_7 as (SELECT * FROM get_date_max where close_7 >= 0),
    get_7 as (SELECT tb7.* FROM cancel_zero_7 tb7 INNER JOIN (SELECT caid,MIN(close_7) 
    minclose_7 FROM cancel_zero_7 GROUP BY caid) tbb7 ON tbb7.caid = tb7.caid 
    WHERE tbb7.minclose_7 = tb7.close_7),
    set_7_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_7
        from get_7),
    
    cancel_zero_8 as (SELECT * FROM get_date_max where close_8 >= 0),
    get_8 as (SELECT tb8.* FROM cancel_zero_8 tb8 INNER JOIN (SELECT caid,MIN(close_8) 
    minclose_8 FROM cancel_zero_8 GROUP BY caid) tbb8 ON tbb8.caid = tb8.caid 
    WHERE tbb8.minclose_8 = tb8.close_8),
    set_8_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_8
        from get_8),
    
    cancel_zero_9 as (SELECT * FROM get_date_max where close_9 >= 0),
    get_9 as (SELECT tb9.* FROM cancel_zero_9 tb9 INNER JOIN (SELECT caid,MIN(close_9) 
    minclose_9 FROM cancel_zero_9 GROUP BY caid) tbb9 ON tbb9.caid = tb9.caid 
    WHERE tbb9.minclose_9 = tb9.close_9),
    set_9_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_9
        from get_9),

    cancel_zero_10 as (SELECT * FROM get_date_max where close_10 >= 0),
    get_10 as (SELECT tb10.* FROM cancel_zero_10 tb10 INNER JOIN (SELECT caid,MIN(close_10) 
    minclose_10 FROM cancel_zero_10 GROUP BY caid) tbb10 ON tbb10.caid = tb10.caid 
    WHERE tbb10.minclose_10 = tb10.close_10),
    set_10_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_10
        from get_10),
    
    cancel_zero_11 as (SELECT * FROM get_date_max where close_11 >= 0),
    get_11 as (SELECT tb11.* FROM  cancel_zero_11 tb11 INNER JOIN (SELECT caid,MIN(close_11) 
    minclose_11 FROM  cancel_zero_11 GROUP BY caid) tbb11 ON tbb11.caid = tb11.caid 
    WHERE tbb11.minclose_11 = tb11.close_11),
    set_11_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_11
        from get_11),
        
    cancel_zero_12 as (SELECT * FROM get_date_max where close_12 >= 0),
    get_12 as (SELECT tb12.* FROM cancel_zero_12 tb12 INNER JOIN (SELECT caid,MIN(close_12) 
    minclose_12 FROM cancel_zero_12 GROUP BY caid) tbb12 ON tbb12.caid = tb12.caid 
    WHERE tbb12.minclose_12 = tb12.close_12),
    set_12_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_12
        from get_12),
        
    cancel_zero_13 as (SELECT * FROM get_date_max where close_13 >= 0),
    get_13 as (SELECT tb13.* FROM cancel_zero_13 tb13 INNER JOIN (SELECT caid,MIN(close_13) 
    minclose_13 FROM cancel_zero_13 GROUP BY caid) tbb13 ON tbb13.caid = tb13.caid 
    WHERE tbb13.minclose_13 = tb13.close_13),
    set_13_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_13
        from get_13),
        
    cancel_zero_14 as (SELECT * FROM get_date_max where close_14 >= 0),
    get_14 as (SELECT tb14.* FROM cancel_zero_14 tb14 INNER JOIN (SELECT caid,MIN(close_14) 
    minclose_14 FROM cancel_zero_14 GROUP BY caid) tbb14 ON tbb14.caid = tb14.caid
    WHERE tbb14.minclose_14 = tb14.close_14),
    set_14_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_14
        from get_14),
        
    cancel_zero_15 as (SELECT * FROM get_date_max where close_15 >= 0),
    get_15 as (SELECT tb15.* FROM cancel_zero_15 tb15 INNER JOIN (SELECT caid,MIN(close_15) 
    minclose_15 FROM cancel_zero_15 GROUP BY caid) tbb15 ON tbb15.caid = tb15.caid 
    WHERE tbb15.minclose_15 = tb15.close_15),
    set_15_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_15
        from get_15),
        
     cancel_zero_16 as (SELECT * FROM get_date_max where close_16 >= 0),
    get_16 as (SELECT tb16.* FROM cancel_zero_16 tb16 INNER JOIN (SELECT caid,MIN(close_16) 
    minclose_16 FROM cancel_zero_16 GROUP BY caid) tbb16 ON tbb16.caid = tb16.caid 
    WHERE tbb16.minclose_16 = tb16.close_16),
    set_16_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_16
        from get_16),

    cancel_zero_17 as (SELECT * FROM get_date_max where close_17 >= 0),
    get_17 as (SELECT tb17.* FROM cancel_zero_17 tb17 INNER JOIN (SELECT caid,MIN(close_17) 
    minclose_17 FROM cancel_zero_17 GROUP BY caid) tbb17 ON tbb17.caid = tb17.caid 
    WHERE tbb17.minclose_17 = tb17.close_17),
    set_17_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_17
        from get_17),
        
    cancel_zero_18 as (SELECT * FROM get_date_max where close_18 >= 0),
    get_18 as (SELECT tb18.* FROM cancel_zero_18 tb18 INNER JOIN (SELECT caid,MIN(close_18) 
    minclose_18 FROM cancel_zero_18 GROUP BY caid) tbb18 ON tbb18.caid = tb18.caid 
    WHERE tbb18.minclose_18 = tb18.close_18),
    set_18_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_18
        from get_18),

    cancel_zero_19 as (SELECT * FROM get_date_max where close_19 >= 0),
    get_19 as (SELECT tb19.* FROM cancel_zero_19 tb19 INNER JOIN (SELECT caid,MIN(close_19) 
    minclose_19 FROM cancel_zero_19 GROUP BY caid) tbb19 ON tbb19.caid = tb19.caid 
    WHERE tbb19.minclose_19 = tb19.close_19),
    set_19_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_19
        from get_19),
        
    cancel_zero_20 as (SELECT * FROM get_date_max where close_20 >= 0),
    get_20 as (SELECT tb20.* FROM cancel_zero_20 tb20 INNER JOIN (SELECT caid,MIN(close_20) 
    minclose_20 FROM cancel_zero_20 GROUP BY caid) tbb20 ON tbb20.caid = tb20.caid 
    WHERE tbb20.minclose_20 = tb20.close_20),
    set_20_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_20
        from get_20),
        
    cancel_zero_21 as (SELECT * FROM get_date_max where close_21 >= 0),
    get_21 as (SELECT tb21.* FROM cancel_zero_21 tb21 INNER JOIN (SELECT caid,MIN(close_21) 
    minclose_21 FROM cancel_zero_21 GROUP BY caid) tbb21 ON tbb21.caid = tb21.caid 
    WHERE tbb21.minclose_21 = tb21.close_21),
    set_21_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_21
        from get_21),

    cancel_zero_22 as (SELECT * FROM get_date_max where close_22 >= 0),
    get_22 as (SELECT tb22.* FROM cancel_zero_22 tb22 INNER JOIN (SELECT caid,MIN(close_22) 
    minclose_22 FROM cancel_zero_22 GROUP BY caid) tbb22 ON tbb22.caid = tb22.caid 
    WHERE tbb22.minclose_22 = tb22.close_22),
    set_22_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_22
        from get_22),
        
    cancel_zero_23 as (SELECT * FROM get_date_max where close_23 >= 0),
    get_23 as (SELECT tb23.* FROM cancel_zero_23 tb23 INNER JOIN (SELECT caid,MIN(close_23) 
    minclose_23 FROM cancel_zero_23 GROUP BY caid) tbb23 ON tbb23.caid = tb23.caid 
    WHERE tbb23.minclose_23 = tb23.close_23),
    set_23_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_23
        from get_23),
        
    cancel_zero_0 as (SELECT * FROM get_date_max where close_0 >= 0),
    get_0 as (SELECT tb0.* FROM cancel_zero_0 tb0 INNER JOIN (SELECT caid,MIN(close_0) 
    minclose_0 FROM cancel_zero_0 GROUP BY caid ) tbb0 ON tbb0.caid = tb0.caid 
    WHERE tbb0.minclose_0 = tb0.close_0),
    set_0_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_0
        from get_0),
        
       union_all as 
        ---选择每个caid每天分别离24个小时最近的记录，也就是每个id的二十四个状态，将其合并，增添一些赋值为xx的空行方便合并
    (select caid, date, number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_0_min
    UNION ALL 
    select caid, date, "xx" as number_0, number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_1_min
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_2_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2,number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_3_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_4_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_5_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_6_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_7_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_8_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_9_min)
    UNION ALL
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_10_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_11_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_12_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_13_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_14_min)
    UNION ALL 
    (select caid, date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_15_min
    UNION ALL 
    select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_16_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_17_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_18_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_19_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_20_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, number_21, "xx" as number_22, "xx" as number_23 from set_21_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, number_22, "xx" as number_23 from set_22_min)
    UNION ALL 
    (select caid,date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, number_23 from set_23_min
    ))
    
 ---计算所有人在所有天里，在0点中分别是在家，在工作和Other的状态，并记录下来，后面同理
    select 
        countif(number_0 = 'home') as num_records_home_0,
        countif(number_0 = 'work') as num_records_work_0, 
        countif(number_0 = 'other') as num_records_other_0,
        countif(number_1 = 'home') as num_records_home_1,
        countif(number_1 = 'work') as num_records_work_1, 
        countif(number_1 = 'other') as num_records_other_1,
        countif(number_2 = 'home') as num_records_home_2,
        countif(number_2 = 'work') as num_records_work_2, 
        countif(number_2 = 'other') as num_records_other_2,
        countif(number_3 = 'home') as num_records_home_3,
        countif(number_3 = 'work') as num_records_work_3, 
        countif(number_3 = 'other') as num_records_other_3,
        countif(number_4 = 'home') as num_records_home_4,
        countif(number_4 = 'work') as num_records_work_4, 
        countif(number_4 = 'other') as num_records_other_4,
        countif(number_5 = 'home') as num_records_home_5,
        countif(number_5 = 'work') as num_records_work_5, 
        countif(number_5 = 'other') as num_records_other_5,
        countif(number_6 = 'home') as num_records_home_6,
        countif(number_6 = 'work') as num_records_work_6, 
        countif(number_6 = 'other') as num_records_other_6,
        countif(number_7 = 'home') as num_records_home_7,
        countif(number_7 = 'work') as num_records_work_7, 
        countif(number_7 = 'other') as num_records_other_7,
        countif(number_8 = 'home') as num_records_home_8,
        countif(number_8 = 'work') as num_records_work_8, 
        countif(number_8 = 'other') as num_records_other_8,
        countif(number_9 = 'home') as num_records_home_9,
        countif(number_9 = 'work') as num_records_work_9, 
        countif(number_9 = 'other') as num_records_other_9,
        countif(number_10 = 'home') as num_records_home_10,
        countif(number_10 = 'work') as num_records_work_10, 
        countif(number_10 = 'other') as num_records_other_10,
        countif(number_11 = 'home') as num_records_home_11,
        countif(number_11 = 'work') as num_records_work_11, 
        countif(number_11 = 'other') as num_records_other_11,
        countif(number_12 = 'home') as num_records_home_12,
        countif(number_12 = 'work') as num_records_work_12, 
        countif(number_12 = 'other') as num_records_other_12,
        countif(number_13 = 'home') as num_records_home_13,
        countif(number_13 = 'work') as num_records_work_13, 
        countif(number_13 = 'other') as num_records_other_13,
        countif(number_14 = 'home') as num_records_home_14,
        countif(number_14 = 'work') as num_records_work_14, 
        countif(number_14 = 'other') as num_records_other_14,
        countif(number_15 = 'home') as num_records_home_15,
        countif(number_15 = 'work') as num_records_work_15, 
        countif(number_15 = 'other') as num_records_other_15,
        countif(number_16 = 'home') as num_records_home_16,
        countif(number_16 = 'work') as num_records_work_16, 
        countif(number_16 = 'other') as num_records_other_16,
        countif(number_17 = 'home') as num_records_home_17,
        countif(number_17 = 'work') as num_records_work_17, 
        countif(number_17 = 'other') as num_records_other_17,
        countif(number_18 = 'home') as num_records_home_18,
        countif(number_18 = 'work') as num_records_work_18, 
        countif(number_18 = 'other') as num_records_other_18,
        countif(number_19 = 'home') as num_records_home_19,
        countif(number_19 = 'work') as num_records_work_19, 
        countif(number_19 = 'other') as num_records_other_19,
        countif(number_20 = 'home') as num_records_home_20,
        countif(number_20 = 'work') as num_records_work_20, 
        countif(number_20 = 'other') as num_records_other_20,
        countif(number_21 = 'home') as num_records_home_21,
        countif(number_21 = 'work') as num_records_work_21, 
        countif(number_21 = 'other') as num_records_other_21,
        countif(number_22 = 'home') as num_records_home_22,
        countif(number_22 = 'work') as num_records_work_22, 
        countif(number_22 = 'other') as num_records_other_22,
        countif(number_23 = 'home') as num_records_home_23,
        countif(number_23 = 'work') as num_records_work_23, 
        countif(number_23 = 'other') as num_records_other_23 from union_all)
    '''  
    
query_job = client.query(query_label, job_config=job_config)
query_job.result() 

In [170]:
#validtaion4，Redding，do not run,5.2日的select

namelist = ['06-29']
for x in namelist:
    
    default_dataset = 'firm-exit.test_chongrui_redding_4_restrict'
    
    job_config = bigquery.QueryJobConfig(default_dataset = default_dataset)

query_label = f'''
    Create or replace table validation_'''+ x + f''' as (
    with check_for_unique as
    (select 
        -- take necessary variables and conversions
        distinct PDT_time, caid, location_name,
        date(PDT_time) as date
        FROM `firm-exit.veraset_tripchain.visitation_ca`),  select_near as
    (select*,
        -- 取前后两天的数据，加上当天一共三天
      from check_for_unique where PDT_time >= '2019-06-28 00:00:00' and PDT_time < '2019-06-30 00:00:00'),
    --计算每个caid在每一次访问中距离每一个整点的时间(这里考虑到跨天的情况)，并取绝对值
    get_date_max as (select caid, date,location_name, PDT_time,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 00:00:00' AS TIMESTAMP),CAST(PDT_time AS TIMESTAMP),MINUTE) as close_0,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 01:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_1,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 02:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_2,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 03:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_3,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 04:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_4,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 05:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_5,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 06:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_6,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 07:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_7,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 08:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_8,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 09:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_9,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 10:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_10,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 11:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_11,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 12:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_12,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 13:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_13,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 14:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_14,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 15:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_15,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 16:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_16,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 17:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_17,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 18:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_18,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 19:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_19,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 20:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_20,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 21:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_21,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 22:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_22,
    TIMESTAMP_DIFF(CAST('2019-'''+ x + f''' 23:00:00' AS TIMESTAMP) ,CAST(PDT_time AS TIMESTAMP),MINUTE) as close_23
    from select_near),
    
    
    cancel_zero_1 as (SELECT * FROM get_date_max where close_1 >= 0),
    get_1 as (SELECT tb1.* FROM cancel_zero_1 tb1 INNER JOIN (SELECT caid,MIN(close_1) 
    minclose_1 FROM cancel_zero_1 GROUP BY caid) tbb1 ON tbb1.caid = tb1.caid 
    WHERE tbb1.minclose_1 = tb1.close_1),
    set_1_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_1
        from get_1),
        
    cancel_zero_2 as (SELECT * FROM get_date_max where close_2 >= 0),
    get_2 as (SELECT tb2.* FROM cancel_zero_2 tb2 INNER JOIN (SELECT caid,MIN(close_2) 
    minclose_2 FROM cancel_zero_2 GROUP BY caid) tbb2 ON tbb2.caid = tb2.caid 
    WHERE tbb2.minclose_2 = tb2.close_2),
    set_2_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_2
        from get_2),
    
    
    cancel_zero_3 as (SELECT * FROM get_date_max where close_3 >= 0),
    get_3 as (SELECT tb3.* FROM cancel_zero_3 tb3 INNER JOIN (SELECT caid,MIN(close_3) 
    minclose_3 FROM cancel_zero_3 GROUP BY caid) tbb3 ON tbb3.caid = tb3.caid 
    WHERE tbb3.minclose_3 = tb3.close_3),
    set_3_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_3
        from get_3),
    
    cancel_zero_4 as (SELECT * FROM get_date_max where close_4 >= 0),
    get_4 as (SELECT tb4.* FROM cancel_zero_4 tb4 INNER JOIN (SELECT caid,MIN(close_4) 
    minclose_4 FROM cancel_zero_4 GROUP BY caid) tbb4 ON tbb4.caid = tb4.caid 
    WHERE tbb4.minclose_4 = tb4.close_4),
    set_4_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_4
        from get_4),
    
    cancel_zero_5 as (SELECT * FROM get_date_max where close_5 >= 0),
    get_5 as (SELECT tb5.* FROM cancel_zero_5 tb5 INNER JOIN (SELECT caid,MIN(close_5) 
    minclose_5 FROM cancel_zero_5 GROUP BY caid) tbb5 ON tbb5.caid = tb5.caid 
    WHERE tbb5.minclose_5 = tb5.close_5),
    set_5_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_5
        from get_5),
    
    cancel_zero_6 as (SELECT * FROM get_date_max where close_6 >= 0),
    get_6 as (SELECT tb6.* FROM cancel_zero_6 tb6 INNER JOIN (SELECT caid,MIN(close_6) 
    minclose_6 FROM cancel_zero_6 GROUP BY caid) tbb6 ON tbb6.caid = tb6.caid 
      WHERE tbb6.minclose_6 = tb6.close_6),
    set_6_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_6
        from get_6),
        
    cancel_zero_7 as (SELECT * FROM get_date_max where close_7 >= 0),
    get_7 as (SELECT tb7.* FROM cancel_zero_7 tb7 INNER JOIN (SELECT caid,MIN(close_7) 
    minclose_7 FROM cancel_zero_7 GROUP BY caid) tbb7 ON tbb7.caid = tb7.caid 
    WHERE tbb7.minclose_7 = tb7.close_7),
    set_7_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_7
        from get_7),
    
    cancel_zero_8 as (SELECT * FROM get_date_max where close_8 >= 0),
    get_8 as (SELECT tb8.* FROM cancel_zero_8 tb8 INNER JOIN (SELECT caid,MIN(close_8) 
    minclose_8 FROM cancel_zero_8 GROUP BY caid) tbb8 ON tbb8.caid = tb8.caid 
    WHERE tbb8.minclose_8 = tb8.close_8),
    set_8_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_8
        from get_8),
    
    cancel_zero_9 as (SELECT * FROM get_date_max where close_9 >= 0),
    get_9 as (SELECT tb9.* FROM cancel_zero_9 tb9 INNER JOIN (SELECT caid,MIN(close_9) 
    minclose_9 FROM cancel_zero_9 GROUP BY caid) tbb9 ON tbb9.caid = tb9.caid 
    WHERE tbb9.minclose_9 = tb9.close_9),
    set_9_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_9
        from get_9),

    cancel_zero_10 as (SELECT * FROM get_date_max where close_10 >= 0),
    get_10 as (SELECT tb10.* FROM cancel_zero_10 tb10 INNER JOIN (SELECT caid,MIN(close_10) 
    minclose_10 FROM cancel_zero_10 GROUP BY caid) tbb10 ON tbb10.caid = tb10.caid 
    WHERE tbb10.minclose_10 = tb10.close_10),
    set_10_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_10
        from get_10),
    
    cancel_zero_11 as (SELECT * FROM get_date_max where close_11 >= 0),
    get_11 as (SELECT tb11.* FROM  cancel_zero_11 tb11 INNER JOIN (SELECT caid,MIN(close_11) 
    minclose_11 FROM  cancel_zero_11 GROUP BY caid) tbb11 ON tbb11.caid = tb11.caid 
    WHERE tbb11.minclose_11 = tb11.close_11),
    set_11_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_11
        from get_11),
        
    cancel_zero_12 as (SELECT * FROM get_date_max where close_12 >= 0),
    get_12 as (SELECT tb12.* FROM cancel_zero_12 tb12 INNER JOIN (SELECT caid,MIN(close_12) 
    minclose_12 FROM cancel_zero_12 GROUP BY caid) tbb12 ON tbb12.caid = tb12.caid 
    WHERE tbb12.minclose_12 = tb12.close_12),
    set_12_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_12
        from get_12),
        
    cancel_zero_13 as (SELECT * FROM get_date_max where close_13 >= 0),
    get_13 as (SELECT tb13.* FROM cancel_zero_13 tb13 INNER JOIN (SELECT caid,MIN(close_13) 
    minclose_13 FROM cancel_zero_13 GROUP BY caid) tbb13 ON tbb13.caid = tb13.caid 
    WHERE tbb13.minclose_13 = tb13.close_13),
    set_13_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_13
        from get_13),
        
    cancel_zero_14 as (SELECT * FROM get_date_max where close_14 >= 0),
    get_14 as (SELECT tb14.* FROM cancel_zero_14 tb14 INNER JOIN (SELECT caid,MIN(close_14) 
    minclose_14 FROM cancel_zero_14 GROUP BY caid) tbb14 ON tbb14.caid = tb14.caid
    WHERE tbb14.minclose_14 = tb14.close_14),
    set_14_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_14
        from get_14),
        
    cancel_zero_15 as (SELECT * FROM get_date_max where close_15 >= 0),
    get_15 as (SELECT tb15.* FROM cancel_zero_15 tb15 INNER JOIN (SELECT caid,MIN(close_15) 
    minclose_15 FROM cancel_zero_15 GROUP BY caid) tbb15 ON tbb15.caid = tb15.caid 
    WHERE tbb15.minclose_15 = tb15.close_15),
    set_15_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_15
        from get_15),
        
     cancel_zero_16 as (SELECT * FROM get_date_max where close_16 >= 0),
    get_16 as (SELECT tb16.* FROM cancel_zero_16 tb16 INNER JOIN (SELECT caid,MIN(close_16) 
    minclose_16 FROM cancel_zero_16 GROUP BY caid) tbb16 ON tbb16.caid = tb16.caid 
    WHERE tbb16.minclose_16 = tb16.close_16),
    set_16_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_16
        from get_16),

    cancel_zero_17 as (SELECT * FROM get_date_max where close_17 >= 0),
    get_17 as (SELECT tb17.* FROM cancel_zero_17 tb17 INNER JOIN (SELECT caid,MIN(close_17) 
    minclose_17 FROM cancel_zero_17 GROUP BY caid) tbb17 ON tbb17.caid = tb17.caid 
    WHERE tbb17.minclose_17 = tb17.close_17),
    set_17_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_17
        from get_17),
        
    cancel_zero_18 as (SELECT * FROM get_date_max where close_18 >= 0),
    get_18 as (SELECT tb18.* FROM cancel_zero_18 tb18 INNER JOIN (SELECT caid,MIN(close_18) 
    minclose_18 FROM cancel_zero_18 GROUP BY caid) tbb18 ON tbb18.caid = tb18.caid 
    WHERE tbb18.minclose_18 = tb18.close_18),
    set_18_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_18
        from get_18),

    cancel_zero_19 as (SELECT * FROM get_date_max where close_19 >= 0),
    get_19 as (SELECT tb19.* FROM cancel_zero_19 tb19 INNER JOIN (SELECT caid,MIN(close_19) 
    minclose_19 FROM cancel_zero_19 GROUP BY caid) tbb19 ON tbb19.caid = tb19.caid 
    WHERE tbb19.minclose_19 = tb19.close_19),
    set_19_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_19
        from get_19),
        
    cancel_zero_20 as (SELECT * FROM get_date_max where close_20 >= 0),
    get_20 as (SELECT tb20.* FROM cancel_zero_20 tb20 INNER JOIN (SELECT caid,MIN(close_20) 
    minclose_20 FROM cancel_zero_20 GROUP BY caid) tbb20 ON tbb20.caid = tb20.caid 
    WHERE tbb20.minclose_20 = tb20.close_20),
    set_20_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_20
        from get_20),
        
    cancel_zero_21 as (SELECT * FROM get_date_max where close_21 >= 0),
    get_21 as (SELECT tb21.* FROM cancel_zero_21 tb21 INNER JOIN (SELECT caid,MIN(close_21) 
    minclose_21 FROM cancel_zero_21 GROUP BY caid) tbb21 ON tbb21.caid = tb21.caid 
    WHERE tbb21.minclose_21 = tb21.close_21),
    set_21_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_21
        from get_21),

    cancel_zero_22 as (SELECT * FROM get_date_max where close_22 >= 0),
    get_22 as (SELECT tb22.* FROM cancel_zero_22 tb22 INNER JOIN (SELECT caid,MIN(close_22) 
    minclose_22 FROM cancel_zero_22 GROUP BY caid) tbb22 ON tbb22.caid = tb22.caid 
    WHERE tbb22.minclose_22 = tb22.close_22),
    set_22_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_22
        from get_22),
        
    cancel_zero_23 as (SELECT * FROM get_date_max where close_23 >= 0),
    get_23 as (SELECT tb23.* FROM cancel_zero_23 tb23 INNER JOIN (SELECT caid,MIN(close_23) 
    minclose_23 FROM cancel_zero_23 GROUP BY caid) tbb23 ON tbb23.caid = tb23.caid 
    WHERE tbb23.minclose_23 = tb23.close_23),
    set_23_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_23
        from get_23),
        
    cancel_zero_0 as (SELECT * FROM get_date_max where close_0 >= 0),
    get_0 as (SELECT tb0.* FROM cancel_zero_0 tb0 INNER JOIN (SELECT caid,MIN(close_0) 
    minclose_0 FROM cancel_zero_0 GROUP BY caid ) tbb0 ON tbb0.caid = tb0.caid 
    WHERE tbb0.minclose_0 = tb0.close_0),
    set_0_min as
    (select *,
      case 
          when(location_name = 'home') then 'home'
          when(location_name = 'work') then 'work'
          else 'other'
        end
        as number_0
        from get_0),
        
       union_all as 
        ---选择每个caid每天分别离24个小时最近的记录，也就是每个id的二十四个状态，将其合并，增添一些赋值为xx的空行方便合并
    (select caid, date, number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_0_min
    UNION ALL 
    select caid, date, "xx" as number_0, number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_1_min
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_2_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2,number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_3_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_4_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_5_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_6_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_7_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_8_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_9_min)
    UNION ALL
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_10_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_11_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_12_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_13_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_14_min)
    UNION ALL 
    (select caid, date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_15_min
    UNION ALL 
    select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_16_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_17_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, number_18, "xx" as number_19,
     "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_18_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_19_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    number_20, "xx" as number_21, "xx" as number_22, "xx" as number_23 from set_20_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
     "xx" as number_20, number_21, "xx" as number_22, "xx" as number_23 from set_21_min)
    UNION ALL 
    (select caid, date, "xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, number_22, "xx" as number_23 from set_22_min)
    UNION ALL 
    (select caid,date,"xx" as number_0, "xx" as number_1, "xx" as number_2, "xx" as number_3, "xx" as number_4, "xx" as number_5, "xx" as number_6,
    "xx" as number_7, "xx" as number_8, "xx" as number_9, "xx" as number_10, "xx" as number_11, "xx" as number_12, "xx" as number_13,
    "xx" as number_14, "xx" as number_15, "xx" as number_16, "xx" as number_17, "xx" as number_18, "xx" as number_19,
    "xx" as number_20, "xx" as number_21, "xx" as number_22, number_23 from set_23_min
    ))
    
 ---计算所有人在所有天里，在0点中分别是在家，在工作和Other的状态，并记录下来，后面同理
    select 
        countif(number_0 = 'home') as num_records_home_0,
        countif(number_0 = 'work') as num_records_work_0, 
        countif(number_0 = 'other') as num_records_other_0,
        countif(number_1 = 'home') as num_records_home_1,
        countif(number_1 = 'work') as num_records_work_1, 
        countif(number_1 = 'other') as num_records_other_1,
        countif(number_2 = 'home') as num_records_home_2,
        countif(number_2 = 'work') as num_records_work_2, 
        countif(number_2 = 'other') as num_records_other_2,
        countif(number_3 = 'home') as num_records_home_3,
        countif(number_3 = 'work') as num_records_work_3, 
        countif(number_3 = 'other') as num_records_other_3,
        countif(number_4 = 'home') as num_records_home_4,
        countif(number_4 = 'work') as num_records_work_4, 
        countif(number_4 = 'other') as num_records_other_4,
        countif(number_5 = 'home') as num_records_home_5,
        countif(number_5 = 'work') as num_records_work_5, 
        countif(number_5 = 'other') as num_records_other_5,
        countif(number_6 = 'home') as num_records_home_6,
        countif(number_6 = 'work') as num_records_work_6, 
        countif(number_6 = 'other') as num_records_other_6,
        countif(number_7 = 'home') as num_records_home_7,
        countif(number_7 = 'work') as num_records_work_7, 
        countif(number_7 = 'other') as num_records_other_7,
        countif(number_8 = 'home') as num_records_home_8,
        countif(number_8 = 'work') as num_records_work_8, 
        countif(number_8 = 'other') as num_records_other_8,
        countif(number_9 = 'home') as num_records_home_9,
        countif(number_9 = 'work') as num_records_work_9, 
        countif(number_9 = 'other') as num_records_other_9,
        countif(number_10 = 'home') as num_records_home_10,
        countif(number_10 = 'work') as num_records_work_10, 
        countif(number_10 = 'other') as num_records_other_10,
        countif(number_11 = 'home') as num_records_home_11,
        countif(number_11 = 'work') as num_records_work_11, 
        countif(number_11 = 'other') as num_records_other_11,
        countif(number_12 = 'home') as num_records_home_12,
        countif(number_12 = 'work') as num_records_work_12, 
        countif(number_12 = 'other') as num_records_other_12,
        countif(number_13 = 'home') as num_records_home_13,
        countif(number_13 = 'work') as num_records_work_13, 
        countif(number_13 = 'other') as num_records_other_13,
        countif(number_14 = 'home') as num_records_home_14,
        countif(number_14 = 'work') as num_records_work_14, 
        countif(number_14 = 'other') as num_records_other_14,
        countif(number_15 = 'home') as num_records_home_15,
        countif(number_15 = 'work') as num_records_work_15, 
        countif(number_15 = 'other') as num_records_other_15,
        countif(number_16 = 'home') as num_records_home_16,
        countif(number_16 = 'work') as num_records_work_16, 
        countif(number_16 = 'other') as num_records_other_16,
        countif(number_17 = 'home') as num_records_home_17,
        countif(number_17 = 'work') as num_records_work_17, 
        countif(number_17 = 'other') as num_records_other_17,
        countif(number_18 = 'home') as num_records_home_18,
        countif(number_18 = 'work') as num_records_work_18, 
        countif(number_18 = 'other') as num_records_other_18,
        countif(number_19 = 'home') as num_records_home_19,
        countif(number_19 = 'work') as num_records_work_19, 
        countif(number_19 = 'other') as num_records_other_19,
        countif(number_20 = 'home') as num_records_home_20,
        countif(number_20 = 'work') as num_records_work_20, 
        countif(number_20 = 'other') as num_records_other_20,
        countif(number_21 = 'home') as num_records_home_21,
        countif(number_21 = 'work') as num_records_work_21, 
        countif(number_21 = 'other') as num_records_other_21,
        countif(number_22 = 'home') as num_records_home_22,
        countif(number_22 = 'work') as num_records_work_22, 
        countif(number_22 = 'other') as num_records_other_22,
        countif(number_23 = 'home') as num_records_home_23,
        countif(number_23 = 'work') as num_records_work_23, 
        countif(number_23 = 'other') as num_records_other_23 from union_all)
    '''  
    
query_job = client.query(query_label, job_config=job_config)
query_job.result() 